In [1]:
import pandas as pd
import glob as glob
import os
import numpy as np

Loading measurement datafiles

In [2]:
paths_remodelling = glob.glob(os.path.join("D:/Turku/microrheometry/2024/lunagel_50s_UV/remodelling", "**","summary_ID_level.csv"),recursive=True)
paths_other = glob.glob(os.path.join("D:/Turku/microrheometry/2024/lunagel_mix_UV", "**","summary_ID_level.csv"),recursive=True)
# print(paths_remodelling, sep='\n')
# print(paths_other, sep='\n')


#cell info
cell_info = (pd.read_csv("D:/Turku/microrheometry/2024/lunagel_50s_UV/remodelling/cell_info.csv")).drop(columns=['Unnamed: 0'])
cell_info['day'] = cell_info['day'].astype(str)
cell_info['experiment'] = cell_info['experiment'].astype(str)

Parsing data

In [9]:
df_remodelling = []
df_other = []
fov_um = 2048*0.325

def parser_remodelling(i):
    data = pd.read_csv(i)
    p= f"lunagel_50s_UV/{(i.split('/')[5])}"
    path_parts = p.split(os.sep)
    path_parts = path_parts[:-2]
    print(os.path.join(*path_parts))
    data['path']= os.path.join(*path_parts)
    data['irradiation'] = (i.split('/')[4].split('_')[1])       
    data['incubation_set'] = i.split('/')[5].split('\\')[1]
    data['incubation'] = i.split('/')[5].split('\\')[2]
    data['experiment'] = i.split('/')[5].split('\\')[3].split('_')[0]
    data['cell/control'] = i.split('/')[5].split('\\')[4].split('_')[-1]
    data['day'] = data['day'].astype(str)
    max_location = data['location'].unique().max()
    data['bead_radius'] = (i.split('/')[5].split('\\')[4].split('_')[1])
    data['location_inverse'] = max_location + 1 - data['location']

    for j, location in enumerate(data['location'].unique()):
        data.loc[data['location'] == location, 'gradient'] = data['y'] + location*fov_um
        data.loc[data['location'] == location, 'gradient_inverse'] = data['y'] + (max_location-location)*fov_um
    return data

def parser_other(i):
    data = pd.read_csv(i)
    p = i.split('/')[4]
    path_parts = p.split(os.sep)
    path_parts = path_parts[:-2]
    data['path']= os.path.join(*path_parts)
    print(os.path.join(*path_parts))
    data['path']=i.split('/')[4]
    data['irradiation'] = (i.split('/')[4].split('\\')[2].split('_')[1])     
    data['incubation_set'] = '0'
    data['incubation'] = '2'
    data['experiment'] = '0'
    data['cell/control'] = (i.split('/')[4].split('\\')[2].split('_')[2])
    data['day'] = data['day'].astype(str)
    max_location = data['location'].unique().max()
    data['bead_radius'] = (i.split('/')[4].split('\\')[2].split('_')[3])
    data['location_inverse'] = max_location + 1 - data['location']

    for j, location in enumerate(data['location'].unique()):
        data.loc[data['location'] == location, 'gradient'] = data['y'] + location*fov_um
        data.loc[data['location'] == location, 'gradient_inverse'] = data['y'] + (max_location-location)*fov_um
    return data

df_remodelling = pd.concat([parser_remodelling(i) for i in paths_remodelling], ignore_index=True)
df_other= pd.concat([parser_other(i) for i in paths_other], ignore_index=True)
merged = pd.concat([df_other, df_remodelling], ignore_index=True)


lunagel_50s_UV/remodelling\0_168\0\0_240823\240823_50um_3layer_50s_control\holder_2
lunagel_50s_UV/remodelling\0_168\0\0_240823\240823_50um_3layer_50s_MDAMB231
lunagel_50s_UV/remodelling\0_168\168\0_240830\240830_50um_3laye_50s_MDAMB231
lunagel_50s_UV/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_control
lunagel_50s_UV/remodelling\24_120\120\0_241111\241111_10um_3layer_50s_MDAMB231
lunagel_50s_UV/remodelling\24_120\120\1_241125\241125_10um_3layers_50s_control
lunagel_50s_UV/remodelling\24_120\120\1_241125\241125_10um_3layers_50s_MDAMB231
lunagel_50s_UV/remodelling\24_120\24\0_241107\241107_10um_3layer_50s_control
lunagel_50s_UV/remodelling\24_120\24\0_241107\241107_10um_3layer_50s_MDAMB231
lunagel_50s_UV/remodelling\24_120\24\1_241121\241121_10um_3layer_50s_control
lunagel_50s_UV/remodelling\24_120\24\1_241121\241121_10um_3layer_50s_MDAMB231
lunagel_50s_UV/remodelling\48_144\144\0_241112\241112_10um_3layer_50s_control
lunagel_50s_UV/remodelling\48_144\144\0_241112\241112_10um_

Direction of the gradient in remodelling (incubation sets) data

In [10]:
merged['gradient_direction'] = np.where(
    ((merged['cell/control'] == "MDAMB231") & (merged['day'] == "241107")) | 
    (merged['day'].isin(["241121","241122", "241125","241126","240830","240905","240913"])),
    "stiff_to_soft",
    "soft_to_stiff")

Parameters for selection of close-by beads


In [11]:
time = 180 #min
speed = 0.5 # or 0.6µm/min
pixel_size = 0.325 #µm
distance = time*speed/pixel_size
print(distance)

276.9230769230769


Assigning to the relevant measurements if cells closeby of faraway

In [12]:
grouped_all = merged.groupby(['day', 'experiment', 'holder', 'location'])
grouped_cell = cell_info.groupby(['day', 'experiment', 'holder', 'location'])

df_remodelling['cell_rel_distance'] = 'unclassified' 

closeby = []
faraway = []

closeby_count = 0
faraway_count = 0

unmatched_keys = set(grouped_all.groups.keys())

for key_all, j_all in grouped_all:
    matched = False 
    for key_cell, j_cell in grouped_cell:
        if key_all == key_cell:
            matched = True 
            unmatched_keys.discard(key_all) 
            # print(f"Processing group: {key_all}")
            for index_all, row_all in j_all.iterrows():
                bead_classified = False
                for index_cell, row_cell in j_cell.iterrows():
                    if ((row_cell['x'] - distance) < row_all['x'] < (row_cell['x'] + distance)) and \
                       ((row_cell['y'] - distance) < row_all['y'] < (row_cell['y'] + distance)):
                        df_remodelling.at[index_all, 'cell_rel_distance'] = 'close'
                        closeby.append(row_all)
                        bead_classified = True
                        closeby_count += 1
                        break
                if not bead_classified:
                    df_remodelling.at[index_all, 'cell_rel_distance'] = 'far'
                    faraway.append(row_all)
                    faraway_count += 1
    if not matched:
        print(f"No matching key_cell for key_all: {key_all}")

final_closeby = pd.DataFrame(closeby)
final_faraway = pd.DataFrame(faraway)

print(df_remodelling['cell_rel_distance'].value_counts())

No matching key_cell for key_all: ('240823', '0', 1, 1)
No matching key_cell for key_all: ('240823', '0', 1, 2)
No matching key_cell for key_all: ('240823', '0', 1, 3)
No matching key_cell for key_all: ('240823', '0', 1, 4)
No matching key_cell for key_all: ('240823', '0', 2, 2)
No matching key_cell for key_all: ('240823', '0', 2, 3)
No matching key_cell for key_all: ('240823', '0', 2, 4)
No matching key_cell for key_all: ('240830', '0', 1, 1)
No matching key_cell for key_all: ('240830', '0', 1, 2)
No matching key_cell for key_all: ('240830', '0', 1, 3)
No matching key_cell for key_all: ('240905', '0', 1, 1)
No matching key_cell for key_all: ('240905', '0', 1, 2)
No matching key_cell for key_all: ('240905', '0', 1, 3)
No matching key_cell for key_all: ('240905', '0', 1, 4)
No matching key_cell for key_all: ('240905', '0', 1, 5)
No matching key_cell for key_all: ('240912', '0', 1, 1)
No matching key_cell for key_all: ('240912', '0', 1, 2)
No matching key_cell for key_all: ('240912', '0'

Combining non- with remodelling data, setting experiment_id

In [13]:
merged['experiment_id'] = merged.groupby(['day', 'incubation_set','cell/control','bead_radius','incubation','experiment','irradiation','holder']).ngroup()

In [14]:
sorted(merged['experiment_id'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

Saving datafiles

In [15]:
saving_path = 'C:/Users/srboval1/OneDrive - Aalto University/Collab/Ivaska_collab/data'

df_remodelling.to_csv(saving_path+'/remodelling_data.csv') 
df_other.to_csv(saving_path+'/non_remodelling_data.csv') 
merged.to_csv(saving_path+'/all_data.csv') 
final_closeby.to_csv(saving_path+'/closeby_beads.csv')
final_faraway.to_csv(saving_path+'/faraway_beads.csv') 
